In [7]:
import pandas as pd
import numpy as np
import math

from sklearn.metrics import ndcg_score, dcg_score
from collections import Counter

## 4. Метрики в рекомендательных системах

### Задание 4.5
Пусть у нас есть реальные оценки, выставленные пользователем, и предсказанные оценки:

Реальные оценки: [2, 4, 1, 1, 1]    
Предсказанные оценки: [2, 5, 2, 3, 1]    
Вычислите коэффициент **NDCG**. Округлите результат до двух знаков после точки-разделителя.

In [8]:
true = np.asarray([[2, 4, 1, 1, 1] ])
relevance = np.asarray([[2, 5, 2, 3, 1]])

ans = ndcg_score(true, relevance)
round(ans,2)

0.97

## 5. Проблема холодного старта и popularity-based model
    
Мы будем работать с набором данных Movie Lens. Он содержит идентификаторы для каждого фильма и пользователя, который его смотрел, а также оценку, которую пользователь поставил фильму. В датасете представлено 25 000 095 оценок фильмов от 162 541 пользователя со шкалой оценок от 0.5 до 5.0.

In [9]:
ratings = pd.read_csv('./data/dst-3.0_mathml_14_5_rating.csv')
movies = pd.read_csv('./data/movie.csv')

In [10]:
df=pd.merge(ratings,movies, how='left',on='movieId')
df.head()

,userId,movieId,rating,timestamp,title,genres
0,1,2,3.5,2005-04-02 23:53:47,Jumanji (1995),Adventure|Children|Fantasy
1,1,29,3.5,2005-04-02 23:31:16,"City of Lost Children, The (Cité des enfants p...",Adventure|Drama|Fantasy|Mystery|Sci-Fi
2,1,32,3.5,2005-04-02 23:33:39,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller
3,1,47,3.5,2005-04-02 23:32:07,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,3.5,2005-04-02 23:29:40,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


Признаки в данных
- userId — id пользователя;
- movieId — id фильма;
- rating — выставленный пользователем рейтинг для фильма;
- timestamp — время выставления рейтинга;
- title — название фильма;
- genres — жанры, к которым относится фильм.

### Задание 5.1
Подсчитайте, сколько раз каждый фильм встречается в наборе данных. Отметьте среди перечисленных ниже фильмов те, что встречаются в топ-5 по популярности.

In [11]:
print(*df.title.value_counts().index[:5],sep='\n')
# cnt = Counter(df.title)
# cnt.most_common(5)

Pulp Fiction (1994)
Forrest Gump (1994)
Shawshank Redemption, The (1994)
Silence of the Lambs, The (1991)
Jurassic Park (1993)


### Задание 5.2
Отлично, мы нашли самые востребованные фильмы. Однако если фильм посмотрело много людей, это ещё не значит, что он им понравился. Чтобы понять, как зритель на самом деле относится к фильму, нужны более чёткие данные. К счастью, в наборе данных Movie Lens есть оценки каждого из зрителей.

- Найдите средний рейтинг для каждого из фильмов.
- Найдите фильмы с наивысшим средним рейтингом.
- Введите в качестве ответа фильм, занимающий последнее место среди фильмов с наивысшим рейтингом, если предварительно отсортировать их по алфавитному порядку.   
    
Впишите только название фильма, без кавычек и без года выхода на экран.

In [12]:
sorted_by_rating = df.groupby('title').rating.mean().sort_values(ascending=False)
top_films_title = sorted_by_rating[sorted_by_rating==5].index
ans = top_films_title.sort_values()[-1]

print(ans)

Yonkers Joe (2008)


### Задание 5.3
В двух предыдущих заданиях вы использовали два самых простых метода для создания неперсонализированных рекомендаций. Однако у них обоих есть свои недостатки: поиск наиболее часто просматриваемых фильмов не учитывает того, насколько фильм нравится аудитории, а поиск среднего рейтинга может вывести в рекомендуемые фильмы малоизвестные специфические картины с одной-двумя оценками.

Чтобы решить эти проблемы, объединим два подхода и будем искать средний рейтинг только для фильмов, которые были оценены более 50 раз.

Сколько таких фильмов?

In [13]:
# (df.movieId.value_counts()>50).sum()
(df.title.value_counts()>50).sum()

# mask = (df.groupby('title').movieId.nunique() > 1)
# df.groupby('title').movieId.unique()[mask]

10472

### Задание 5.4
Постройте простейшую рекомендацию: возьмите фильмы, которые смотрели более 50 раз, и найдите среди них фильм с наивысшей оценкой. В качестве ответа запишите название этого фильма без артикля и года выхода на экран.
Если название фильма состоит из двух и более слов, между отдельными словами должен быть только один пробел.

In [14]:
pop_films = df.title.value_counts().index[df.title.value_counts()>50].tolist()
pop_films_mask = df.title.isin(pop_films)
df[pop_films_mask].groupby('title').rating.mean().idxmax()

'Shawshank Redemption, The (1994)'

## 6. Практика
В этом юните мы построим систему рекомендаций, основываясь именно на этом методе. Вы сможете усовершенствовать её в следующем модуле, после того как освоите другие алгоритмы.   
    
Для начала загрузим датасет ["Articles sharing and reading from CI&T DeskDrop"](https://www.kaggle.com/datasets/gspmoreira/articles-sharing-reading-from-cit-deskdrop), включающий в себя собранные за один год логи ***DeskDrop*** — платформы для внутренних коммуникаций, разработанной ***CI&T*** и ориентированной на компании, использующие ***Google Workspace*** (***Google G Suite***). Среди прочего, эта платформа позволяет сотрудникам компаний делиться актуальными статьями со своими коллегами.

В датасете содержится около 73 тысяч записей о взаимодействии пользователей с более чем тремя тысячами публичных статей, размещённых на платформе.

Информация в наборе данных:
- Оригинальный URL, название и текст статьи.
- Контекст посещений пользователей, например дата/время, клиент (мобильное приложение/браузер) и геолокация.
- Различные типы взаимодействия, что позволяет сделать вывод об уровне заинтересованности пользователя в статьях, например комментарии → лайки → просмотры.    
    
Данные включают в себя два файла:   
   
[shared_articles.csv](https://lms.skillfactory.ru/assets/courseware/v1/9f0e8eb4ddd03415fdd4db4a89a2b0d3/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/shared_articles.zip);   
[users_interactions.csv.](https://lms.skillfactory.ru/assets/courseware/v1/186647c8bd3fdb43b78fbc84ace97aed/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/users_interactions.zip)    
     

Начнём работать с файлом `shared_articles.csv`. Он содержит информацию о статьях, опубликованных на платформе ***DeskDrop***.

Для каждой статьи есть:
* дата публикации (временная метка),
* исходный URL-адрес,
* заголовок,
* содержание в виде обычного текста,
* язык статьи (португальский — pt или английский — en),
* информация о пользователе, который поделился статьёй (автор).   
    
Для временной метки существует два возможных типа событий:   
CONTENT SHARED — статья была опубликована на платформе и доступна для пользователей;
CONTENT REMOVED — статья была удалена с платформы и недоступна для дальнейших рекомендаций.   
    
Для простоты мы рассматриваем здесь только тип события **CONTENT SHARED**.

In [15]:
df_articles = pd.read_csv("./data/shared_articles.csv")
df_interactions = pd.read_csv("./data/users_interactions.csv")

### Задание 6.1
Отфильтруйте данные так, чтобы остались только объекты с типом события CONTENT SHARED. Сколько таких объектов в получившейся таблице?

In [16]:
shared_mask = df_articles.eventType == 'CONTENT SHARED'
df_shar_articles = df_articles[shared_mask]

df_shar_articles.shape[0]

3047

Теперь откроем второй файл — `users_interactions.csv`.

В колонке `eventType` описаны действия, которые могли совершать пользователи при взаимодействии со статьёй:
* VIEW — просмотр,
* LIKE — лайк,
* COMMENT CREATED — комментарий,
* FOLLOW — подписка,
* BOOKMARK — добавление в закладки.   
    
В первую очередь нам необходимо понять, как определить, что какая-то статья популярнее других. Если бы из возможных реакций у нас были только лайки или только просмотры, то статьи было бы легко ранжировать в соответствии с этими значениями. Однако у нас есть информация о различных действиях пользователя, и на её основе мы должны создать некий универсальный индекс популярности. Составим его из реакций пользователей, придав им разные веса:
```python
event_type = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}
```
Веса здесь подобраны исходя из важности каждого действия: оставить комментарий — значит, показать наибольшую вовлечённость, а обычный просмотр, напротив, демонстрирует наименьшую вовлечённость.

### Задание 6.2
Создайте признак, который будет отражать числовой вес для взаимодействия со статьёй (в соответствии с приведёнными выше весами). Вычислите среднее значение для полученного признака. Округлите его до двух знаков после точки-разделителя.

In [17]:
event_type = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}

df_interactions['weighted_eventType'] = df_interactions.eventType.map(event_type)
ans = df_interactions.weighted_eventType.mean()
round(ans,2)

1.24

Ранее мы говорили, что рекомендательные системы подвержены проблеме холодного старта — в таких случаях создавать рекомендации намного сложнее.

### Задание 6.3
Чтобы получить хоть какую-то информацию, на которую можно будет опираться, оставьте только тех пользователей, которые взаимодействовали хотя бы с пятью статьями. Сколько всего таких пользователей?

In [18]:
activity_mask = df_interactions.groupby('personId').contentId.nunique()>4
active_person = df_interactions.groupby('personId').contentId.count().index[activity_mask].to_list()

len(active_person)

1140

### Задание 6.4
Теперь оставим только те взаимодействия, которые касаются только отфильтрованных пользователей (то есть тех, которые взаимодействовали как минимум с пятью статьями). Сколько всего таких взаимодействий?

In [19]:
df_interactions = df_interactions[df_interactions.personId.isin(active_person)]

df_interactions.shape[0]

69868

Сейчас каждое отдельное взаимодействие пользователя со статьёй выделено в отдельную запись, то есть пользователь мог просмотреть статью, лайкнуть и прокомментировать её, и всё это отразилось в трёх действиях. Давайте для удобства соединим все эти действия в некоторый коэффициент, который будет отражать интерес пользователя к статье. Так как каждому возможному действию мы ранее уже присвоили вес, то, по сути, нам нужно просто сложить все действия. Однако полученное число будет увеличиваться с количеством действий, и будет очень большой разброс возможных значений. В таких случаях обычно логарифмируют полученный результат с помощью следующей функции:
```python
def smooth_user_preference(x):
    return math.log(1+x, 2)
```

### Задание 6.5
Примените упомянутое выше преобразование для логарифмирования к сумме весов для взаимодействия пользователя с каждой конкретной статьёй. Также сохраните для каждой пары «пользователь — статья» значение времени последнего взаимодействия.

Найдите среднее по признаку с получившимися временными отсечками. Округлите результат до двух знаков после точки-разделителя.
>Чтобы выбрать последнее взаимодействие, необходимо использовать метод `last()`.

>В этом задании необходимо работать с данными, полученными в задании 6.4.

In [20]:
def smooth_user_preference(x):
    return math.log(1+x, 2)

cum_weighted_event = df_interactions.groupby(['personId','contentId']).\
                        weighted_eventType.sum().apply(smooth_user_preference).reset_index()

last_interaction = df_interactions.groupby(['personId','contentId']).timestamp.last().\
                        reset_index().rename(columns={'timestamp':'last_timestamp'})

cum_weighted_event = cum_weighted_event.merge(last_interaction,on=['personId','contentId'])

ans = cum_weighted_event.last_timestamp.mean()
round(ans,2)

1470587338.35

Разумеется, для того чтобы впоследствии оценить качество построенной рекомендательной системы, нам нужно разделить выборку на обучающую и тестовую. Так как в реальности рекомендации строятся на основе исторических данных о пользователе и контенте, сделаем в нашей задаче разбиение на обучающую и тестовую выборки по временной отсечке.   

### Задание 6.6
Разделите данные на обучающую и тестовую выборки, выбрав в качестве временной отсечки значение `1475519545`. Значение отсечки включите в тестовую выборку. Сколько объектов попало в обучающую выборку?

In [29]:
split_timestamp = 1475519545
spli_mask = cum_weighted_event.last_timestamp < split_timestamp

train = cum_weighted_event[spli_mask].copy()
test = cum_weighted_event[~spli_mask].copy()

train.shape[0]

29329

Для удобства дальнейшего измерения качества рекомендаций преобразуйте данные так, чтобы получить таблицу в формате, где строка соответствует пользователю, а столбцы будут истинными предпочтениями и рекомендациями в формате списков. На место пустых ячеек поместите пустые списки.
```python
final_df = (
    train_df.reset_index()
    .groupby('personId')['contentId'].agg(lambda x: list(x))
    .reset_index()
    .rename(columns={'contentId': 'true_train'})
    .set_index('personId')
)

final_df['true_test'] = (
    test_df.reset_index()
    .groupby('personId')['contentId'].agg(lambda x: list(x))
)

final_df['true_test'] = [ [] if x is np.NaN else x for x in final_df['true_test'] ]
```

In [143]:
true_train = train.groupby('personId').contentId.apply(lambda x: list(x))
true_test = test.groupby('personId').contentId.apply(lambda x: list(x))

final_df = pd.DataFrame(data = {'true_train':true_train,'true_test':true_test},
                                 index = true_train.index)
final_df.true_test = final_df.true_test.apply(lambda x: x if x==x else [])

### Задание 6.7
Осталось совсем немного — скоро вы получите свою первую систему рекомендаций! Мы будем строить *popular-based*-модель, а значит, нам необходимо найти самые популярные статьи.

Посчитайте популярность каждой статьи как сумму всех логарифмических «оценок» взаимодействий с ней (используя только обучающую выборку). Выберите ID самой популярной статьи:

In [75]:
content_popularity = train.groupby('contentId').weighted_eventType.sum()
content_popularity.idxmax()

-6783772548752091658

Теперь необходимо сформировать рекомендации для каждого пользователя. Будем рекомендовать десять самых популярных статей. Также необходимо помнить, что следует предлагать пользователю только то, что он ещё не читал.

### Задание 6.8
Постройте систему рекомендаций. Оцените качество с помощью `precision@10` для каждого пользователя (доля угаданных рекомендаций). После этого усредните результат по всем пользователям.   
    
Для вычисления `precision@10` воспользуйтесь следующей функцией:

```python
def precision(column):
    return (final_df.apply(lambda row:
            len(set(row['true_test']).intersection(set(row[column]))) /
            min(len(row['true_test']) + 0.001, 10.0),axis=1)).mean()
```
Итоговый результат округлите до трёх знаков после точки-разделителя.

>В этом задании необходимо использовать данные, полученные в задании 6.6 (для которых приведён скриншот выше).

In [149]:
def calc_precision(column):
    return (
        final_df
        .apply(
            lambda row:
            len(set(row['true_test']).intersection(
                set(row[column]))) /
            min(len(row['true_test']) + 0.001, 10.0),
            axis=1)).mean()

In [151]:
n_top = 10
ranged_content = content_popularity.sort_values(ascending=False).index
unviewed_mask = lambda x: ~ranged_content.isin(list(x))

final_df['recommended'] = final_df.true_train.apply(
    lambda x: ranged_content[unviewed_mask(x)][:n_top].tolist())

ans = calc_precision('recommended')
round(ans,3)

0.006